# TOC
[I. Improve Performance with Ensembles](#I.-Improve-Performance-with-Ensembles)<br><br>
[1.1 Bagging Algorithms](#1.1-Bagging-Algorithms)<br>
[1.1.1 Bagged Decision Trees](#1.1.1-Bagged-Decision-Trees)<br>
[1.1.2 Random Forest](#1.1.2-Random-Forest)<br>
[1.1.3 Extra Trees](#1.1.3-Extra-Trees)<br><br>
[1.2 Boosting Algorithms](#1.2-Boosting-Algorithms)<br>
[1.2.1 AdaBoost](#1.2.1-AdaBoost)<br>
[1.2.2 Stochastic Gradient Boosting](#1.2.2-Stochastic-Gradient-Boosting)<br><br>
[1.3 Voting Ensemble](#1.3-Voting-Ensemble)<br><br>
[II. Improve Performance with Algorithm Tuning](#II.-Improve-Performance-with-Algorithm-Tuning)<br>
[2.1 Machine Learning Algorithm Parameters](#2.1-Machine-Learning-Algorithm-Parameters)<br>
[2.2 Grid Search Parameter Tuning](#2.2-Grid-Search-Parameter-Tuning)

### I. Improve Performance with Ensembles
Sử dụng ensembles để boost accuracy trên tập dữ liệu của bạn.<br>
Kết hợp nhiều mô hình vào các tiên đoán ensemble. 3 phương thức phổ biến nhất cho việc kết hợp các tiên đoán từ các mô hình khác nhau là:
- Bagging: xây dựng nhiều mô hình (lý tưởng là cùng một loại) từ những subsample khác nhau của tập train.
- Boosting: xây dựng nhiều mô hình (lý tưởng là cùng một loại), mỗi cái trong số đó học để fix sai số trong tiên đoán của mô hình trước trong chuỗi các mô hình.
- Voting: xây dựng nhiều mô hình (lý tưởng là khác loại) và những thống kê đơn giản (ví dụ như tính mean) được sử dụng để kết hợp các tiên đoán.

### 1.1 Bagging Algorithms
Bagging liên quan đến việc lấy nhiều mẫu từ bộ dữ liệu train (với sự thay thế) và train một mô hình trên từng mẫu đó. Kết quả cuối cùng là trung bình các tiên đoán của tất cả các mô hình con.
- Bagged Decision Trees.
- Random Forest.
- Extra Trees.

### 1.1.1 Bagged Decision Trees

Bagging thể hiện tốt nhất với những thuật toán có variance cao. Một ví dụ phổ biến nhất là Decision Tree, thường được cấu trúc mà chưa làm gọn. Ở ví dụ bên dưới **100** cây đã được tạo.

In [9]:
import pandas as pd
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv('pima-indians-diabetes.csv', names=names)
array = dataframe.values
X = array[:, :8] # input
Y = array[:, 8] # output
dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

num_folds = 10
seed = 7
scoring = 'accuracy'
kfold = KFold(n_splits=num_folds, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.770745044429255


Sau khi chạy ví dụ trên, ta thấy kết quả mô hình theo độ chính xác đo được khá tốt.

### 1.1.2 Random Forest

Random Forest là phần mở rộng của bagged decision trees. Các mẫu của bộ dữ liệu train vẫn được lấy bằng cách thay thế Nhưng các cây được cấu trúc theo cách giảm mối tương quan giữa các cá thể phân loại. Cụ thể, thay vì chọn một cách tham lam split point tốt nhất trong tất cả các feature thuộc cấu trúc từng cây, thì chỉ có **một bộ** các feature **ngẫu nhiên** được dùng cho việc tách.<br>
Bên dưới là ví dụ cho **100 cây** với bộ subset các feature ngẫu nhiên là **3**.

In [11]:
from sklearn.ensemble import RandomForestClassifier

num_folds = 10
seed = 7
scoring = 'accuracy'
kfold = KFold(n_splits=num_folds, random_state=seed)
num_trees = 100
max_features = 3
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7733766233766234


### 1.1.3 Extra Trees

Extra Trees là một sửa đổi từ bagging. Ở đó những cây ngẫu nhiên thì được cấu trúc từ các mẫu của bộ dữ liệu train.<br>
Ví dụ bên dưới cho **100 cây** và các điểm tách từ **7** features ngẫu nhiên.

In [12]:
from sklearn.ensemble import ExtraTreesClassifier

num_folds = 10
seed = 7
scoring = 'accuracy'
kfold = KFold(n_splits=num_folds, random_state=seed)
num_trees = 100
max_features = 7
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.762987012987013


### 1.2 Boosting Algorithms

Các thuật toán Boosting ensemble tạo ra một chuỗi các mô hình, trong đó chúng sẽ cố sửa chữa những cái sai của mô hình trước đó theo chuỗi. Một khi được tạo ra, các mo hình tạo ra các tiên đoán được trọng số bởi độ chính xác của chúng và các kết quả này được kết hợp lại để tạo ra tiên đoán cuối cùng cho output.<br>
Hai thuật toán boosting ensemble phổ biến nhất là :
- AdaBoost.
- Stochastic Gradient Boosting

### 1.2.1 AdaBoost

AdaBoost có lẽ là thuật toán boosting ensemble thành công. Nó nhìn chung hoạt động bằng cách trọng số các thực thể trong bộ dữ liệu bằng việc chúng khó hoặc dễ phân loại như thế nào. Cho phép thuật toán này hoặc quan tâm hơn hoặc ít quan tâm hơn trong cấu trúc của các mô hình tiếp theo.<br>
Ví dụ bên dưới cấu trúc **30 cây decision trees**.

In [13]:
from sklearn.ensemble import AdaBoostClassifier

num_folds = 10
seed = 7
scoring = 'accuracy'
kfold = KFold(n_splits=num_folds, random_state=seed)
num_trees = 30
cart = DecisionTreeClassifier(max_depth=1)
model = AdaBoostClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
# nếu không chọn base estimator thì mặc định là DecisionTreeClassifier(max_depth=1)
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.760457963089542


### 1.2.2 Stochastic Gradient Boosting

Stochastic Gradient Boosting là một trong những kĩ thuật ensemble phức tạp nhất. Nó cũng là một kĩ thuật được chứng minh là một trong những kĩ thuật tốt nhất cho việc cải thiện khả năng biểu diễn của mô hình qua ensemble.
Ví dụ bên dưới sử dụng Stochastic Gradient Boosting với **100 cây decision trees**.

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
num_folds = 10
seed = 7
scoring = 'accuracy'
kfold = KFold(n_splits=num_folds, random_state=seed)
num_trees = 100
cart = DecisionTreeClassifier(max_depth=1)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed, init=cart)
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7733937115516063


### 1.3 Voting Ensemble

Voting là một trong những cách đơn giản nhất kết hợp các tiên đoán từ nhiều thuật toán machine learning. Nó hoạt động bằng việc tạo ra 2 hay nhiều mô hình chuẩn từ bộ dữ liệu train. Một Voting Classifier có thể dùng để gói các mô hình và trung bình các tiên đoán từ các mô hình con và từ đó tạo ra các tiên đoán cho dữ liệu chưa được nhìn thấy.<br>
Các tiên đoán của các mô hình có thể trọng số nhưng cụ thể trọng số từng mô hình đó thì khó. Một trong những phương thức nâng cao hơn là học cách làm sao để trọng số tốt nhất cho các tiên đoán từ các mô hình con. Cái này được gọi là stacking và trong tài liệu này thì scikit-learn vẫn chưa hỗ trợ.

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

num_folds = 10
seed = 7
scoring = 'accuracy'
kfold = KFold(n_splits=num_folds, random_state=seed)
# create the sub models
estimators = []
estimators.append(('LR', LogisticRegression(solver='liblinear'))) # model 1
estimators.append(('CART', DecisionTreeClassifier(max_depth=1))) # model 2
estimators.append(('SVM', SVC(gamma='auto'))) # model 3
# create the ensemble model
ensemble = VotingClassifier(estimators=estimators)
results = cross_val_score(ensemble, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

0.7656015037593986


### II. Improve Performance with Algorithm Tuning

Các mô hình machine learning thì có những tham số cụ thể mà hành vi của chúng có thể được điều chỉnh cho một bài toán cụ thể. Các mô hình có thể có nhiều tham số và tìm ra được bộ tham số kết hợp tốt nhất thì giải quyết được bài toán tốt hơn.

### 2.1 Machine Learning Algorithm Parameters

Điều chỉnh thuật toán là bước cuối cùng trong quá trình ứng dụng machine learning trước khi hoàn tất mô hình của bạn. Nó thỉnh thoảng được gọi là tối ưu hóa hyperparameter, ở đây thông số của thuật toán được hiểu là hyperparameter. Còn các hệ số được tìm ra trong thuật toán machine learning là thông số<br>
Tối ưu hóa là phương pháp thử-sai cho bài toán đặt ra. Có 2 phương thức đơn giản cho việc điều chỉnh thông số thuật toán là:
- Grid Search Parameter Tuning.
- Random Search Parameter Tuning.

### 2.2 Grid Search Parameter Tuning

Grid search là một cách tiếp cận điều chỉnh tham số mà ở đó sẽ xây dựng và đánh giá **1** mô hình cho từng tham số kết hợp trong grid.

In [16]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV

alphas = [1,0.1,0.01,0.001,0.0001,0] # test with alpha set
class_weights = [{1: 0.7, 0:0.3}, {1:0.5, 0:0.5}, {1:0.4, 0:0.6}] # test with class weight set
param_grid = dict(alpha=alphas, class_weight = class_weights)
model = RidgeClassifier()
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid.fit(X,Y)
print(grid.best_estimator_)
print(grid.best_params_ )
# test with new parameter
results = cross_val_score(grid.best_estimator_, X, Y, cv=kfold, scoring=scoring)
print(results.mean())

RidgeClassifier(alpha=0.1, class_weight={0: 0.5, 1: 0.5}, copy_X=True,
                fit_intercept=True, max_iter=None, normalize=False,
                random_state=None, solver='auto', tol=0.001)
{'alpha': 0.1, 'class_weight': {1: 0.5, 0: 0.5}}
0.7708646616541354


Vậy sau khi grid search cv trên bộ tham số alpha và class_weight thì ta tìm được 'alpha': 0.1, 'class_weight': {1: 0.5, 0: 0.5} là những tham số tốt nhất _trong bộ tham số_ ở mô hình này.